In [1]:
#Import the required packages
#Import package pandas for data analysis
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import datetime as dt

from datetime import datetime, timedelta

from datetime import datetime
from matplotlib import pyplot as plt

#For showing plots directly in the notebook run the command below
%matplotlib inline

from patsy import dmatrices
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA, KernelPCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# For saving multiple plots into a single pdf file
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.pipeline import Pipeline, make_pipeline
# Gets the public holidays for Ireland
import holidays

## Route Selection parameters

In [2]:
RouteNum = '7' 
RouteTrips = f'{RouteNum}/{RouteNum}_CleanedTripsData.csv'
RouteLeavetimes = f'{RouteNum}/{RouteNum}_CleanedLeavetimes.csv'
#RouteTrips = f'Route_{RouteNum}_CleanedTripsData.csv'
#RouteLeavetimes = f'Route_{RouteNum}_CleanedLeavetimes.csv'

# Merged Trips and Leavetimes Datasframes

In [3]:
# Reading the Trips csv file, into a data frame
df_Trips = pd.read_csv(RouteTrips)
# Drop the LINEID
df_Trips.drop(columns=['PLANNEDTIME_ARR','PLANNEDTIME_DEP','ACTUALTIME_ARR', 'ACTUALTIME_DEP'], inplace=True)
# Show data frame first few rows
df_Trips.head()

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION
0,2018-02-07,6259785,7,7_70,2
1,2018-02-18,6269898,7,7_66,1
2,2018-03-11,6386003,7,7_72,1
3,2018-03-11,6394364,7,7_72,1
4,2018-03-11,6397973,7,7_72,1


In [4]:
# Reading the Leavetimes csv file, into a data frame
df_Leavetimes = pd.read_csv(RouteLeavetimes)
# Show data frame first few rows
df_Leavetimes.head()

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,ARR/DEP_PLAN,ACTUALTIME_ARR,ACTUALTIME_DEP
0,2018-01-01,5968841,1,4962,48000,47991,47991
1,2018-01-01,5962093,1,4962,40200,40177,40177
2,2018-01-01,5960117,1,4962,37800,37791,37791
3,2018-01-01,5960605,1,4962,44100,44090,44090
4,2018-01-01,5968653,1,4962,42000,40128,40128


In [5]:
# Merge the Trips and Leavetimes CSV files to a single database df_merged_TripsLeaveTimes
df_merged_TripsLeaveTimes = pd.merge(df_Leavetimes,df_Trips, on=['DAYOFSERVICE','TRIPID'])

In [6]:
# Show the first few rows of the Merged Dataframe
df_merged_TripsLeaveTimes

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,ARR/DEP_PLAN,ACTUALTIME_ARR,ACTUALTIME_DEP,LINEID,ROUTEID,DIRECTION
0,2018-01-01,5968841,1,4962,48000,47991,47991,7,7_66,1
1,2018-01-01,5968841,3,273,48346,48305,48505,7,7_66,1
2,2018-01-01,5968841,4,405,48612,48628,48746,7,7_66,1
3,2018-01-01,5968841,5,408,48748,48825,48840,7,7_66,1
4,2018-01-01,5968841,7,410,48803,48917,48917,7,7_66,1
...,...,...,...,...,...,...,...,...,...,...
1296647,2018-12-31,8586900,51,3218,73802,73197,73209,7,7_73,1
1296648,2018-12-31,8586900,52,3219,73860,73274,73288,7,7_73,1
1296649,2018-12-31,8586900,53,3220,73923,73334,73334,7,7_73,1
1296650,2018-12-31,8586900,54,5046,74100,73421,73421,7,7_73,1


In [7]:
# Separate the dataframe for direction 1
df_merged_TripsLeaveTimes.drop(columns=['LINEID'], inplace= True)


In [8]:
df_merged_TripsLeaveTimes

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,ARR/DEP_PLAN,ACTUALTIME_ARR,ACTUALTIME_DEP,ROUTEID,DIRECTION
0,2018-01-01,5968841,1,4962,48000,47991,47991,7_66,1
1,2018-01-01,5968841,3,273,48346,48305,48505,7_66,1
2,2018-01-01,5968841,4,405,48612,48628,48746,7_66,1
3,2018-01-01,5968841,5,408,48748,48825,48840,7_66,1
4,2018-01-01,5968841,7,410,48803,48917,48917,7_66,1
...,...,...,...,...,...,...,...,...,...
1296647,2018-12-31,8586900,51,3218,73802,73197,73209,7_73,1
1296648,2018-12-31,8586900,52,3219,73860,73274,73288,7_73,1
1296649,2018-12-31,8586900,53,3220,73923,73334,73334,7_73,1
1296650,2018-12-31,8586900,54,5046,74100,73421,73421,7_73,1


In [14]:
# Save cleaned data to a csv
#df_merged_TripsLeaveTimes.to_csv(f'{RouteNum}/{RouteNum}_cleanCombinedCSV.csv', index_label=False)
#df_merged_TripsLeaveTimes.to_csv(f'Route_{RouteNum}_cleanCombinedCSV.csv', index_label=False)

## Move to the General Feature Creation Notebook